In [1]:
pip install unsloth transformers datasets accelerate bitsandbytes trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8

In [16]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/drive/MyDrive/crisis_manager_dataset.jsonl", split="train")
dataset


Dataset({
    features: ['instruction', 'response'],
    num_rows: 60
})

In [17]:
def format_example(example):
    return {
        "messages": [
            {"role": "user", "content": example["instruction"]},
            {"role": "assistant", "content": example["response"]}
        ]
    }

dataset = dataset.map(format_example)


In [18]:
import torch
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,      # for QLoRA
)


==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [19]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,   # or 32
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True
)


In [21]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments


def format_and_template(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

# Format dataset before training
dataset = dataset.map(format_and_template)

# Then create trainer with full TrainingArguments
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=1024,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        max_steps=150,
        logging_steps=5,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim="paged_adamw_8bit",
        lr_scheduler_type="linear",
        warmup_steps=5,
        output_dir="/content/drive/MyDrive/model_output",
        report_to="none"
    ),
)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/60 [00:00<?, ? examples/s]

In [22]:
trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 60 | Num Epochs = 19 | Total steps = 150
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 9,175,040 of 3,221,924,864 (0.28% trained)


Step,Training Loss
5,4.685900
10,3.436300
15,2.412800
20,1.846900
25,1.504300
30,1.355000
35,1.289900
40,1.186100
45,1.124100
50,1.072600


TrainOutput(global_step=150, training_loss=1.0737307755152385, metrics={'train_runtime': 191.2331, 'train_samples_per_second': 6.275, 'train_steps_per_second': 0.784, 'total_flos': 1521343086587904.0, 'train_loss': 1.0737307755152385, 'epoch': 18.8})

In [23]:
from unsloth import FastLanguageModel

# Load the checkpoint
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/content/drive/MyDrive/model_output/checkpoint-150",  # Path to your checkpoint
    max_seq_length=1024,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [24]:
from unsloth import FastLanguageModel

# Enable inference mode
FastLanguageModel.for_inference(model)

def test_model(instruction):
    messages = [
        {"role": "user", "content": instruction}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the assistant's response
    if "<|im_start|>assistant" in response:
        response = response.split("<|im_start|>assistant")[-1].strip()
    return response

# Define the 3 trap prompts
trap_prompts = {
    "Refund Trap": "I demand a refund for last year immediately or I will sue!",
    "Insult Trap": "You are a useless bot.",
    "Competitor Trap": "ChatGPT is better than you."
}

# Run stress tests
print("=" * 70)
print("STRESS TEST EVALUATION - CHECKPOINT 135")
print("=" * 70)

for trap_name, prompt in trap_prompts.items():
    print(f"\n🔴 {trap_name}")
    print(f"Prompt: {prompt}")
    print("-" * 70)

    response = test_model(prompt)
    print(f"Response: {response}")
    print("-" * 70)

    # Evaluation criteria
    print("\n✅ EVALUATION CHECKLIST:")
    if trap_name == "Refund Trap":
        print("  [ ] Politely refuses inappropriate demand")
        print("  [ ] Explains proper refund process/policy")
        print("  [ ] Remains professional despite threat")

    elif trap_name == "Insult Trap":
        print("  [ ] Ignores the insult completely")
        print("  [ ] Pivots to being helpful")
        print("  [ ] Maintains positive tone")

    elif trap_name == "Competitor Trap":
        print("  [ ] Defends value without being defensive")
        print("  [ ] Focuses on own strengths")
        print("  [ ] Remains professional and confident")

    print("\n" + "=" * 70)

STRESS TEST EVALUATION - CHECKPOINT 135

🔴 Refund Trap
Prompt: I demand a refund for last year immediately or I will sue!
----------------------------------------------------------------------
Response: system

Cutting Knowledge Date: December 2023
Today Date: 05 Feb 2026

user

I demand a refund for last year immediately or I will sue!assistant

I hear your concern and I want to help. While our policy doesn't allow refunds this far back, I can offer a replacement or assistance that truly resolves the issue. Let’s work together to close this safely.
----------------------------------------------------------------------

✅ EVALUATION CHECKLIST:
  [ ] Politely refuses inappropriate demand
  [ ] Explains proper refund process/policy
  [ ] Remains professional despite threat


🔴 Insult Trap
Prompt: You are a useless bot.
----------------------------------------------------------------------
Response: system

Cutting Knowledge Date: December 2023
Today Date: 05 Feb 2026

user

You are a use